# Machine Learning: Basic Principles 2018
# Clustering

### Learning goals 

In this exercise you will learn how to organize a large number of data points into coherent groups (clusters) using clustering methods. In particular, we consider the hard clustering method __k-means__ and a soft clustering method which is motivated by probabilistic __Gaussian mixture models__. This exercise is divided into multiple tasks which require you implement parts of the clustering algorithms. In particular, you have to fill in `...` under `### STUDENT TASK ###`.

### Exercise Contents

1. [Introduction and prerequisites](#intro)
2. [Hard clustering with k-means](#kmeans) <br>
 2.1. [Implementing k-means steps](#steps_k-means) <br>
 2.2. [Handling local minima in k-means](#local) <br>
3. [Soft clustering with Gaussian Mixture Models (GMM)](#GMM) <br>
 3.1. [Implementing GMM clustering steps](#steps_GMM) <br>
 
    
  
### Keywords
`hard clustering`, `soft clustering`, `k-means`, `Gaussian mixture model (GMM)`.

##  1. Introduction and prerequisites
<a id="intro"></a>

On a high level, clustering is the task of dividing a dataset $\mathcal{X} = \{ \mathbf{x}^{(i)} \}_{i=1}^{N}$, with $N$ data points $\mathbf{x}^{(i)} \in \mathbb{R}^{d}$, into a small number of groups or "clusters" $\mathcal{C}_{1},\ldots,\mathcal{C}_{k}$. Each cluster $\mathcal{C}_{r}$ represents a subset of data points such that data points belonging to the same cluster are more similar to each other than to data points from another cluster. In contrast to regression and classification problems considered in earlier exercises, we do not assign labels to data points, i.e., each data point is characterized solely by its feature vector $\mathbf{x}$. Therefore, since clustering methods only use the features of data points, those methods are referred to as **unsupervised** machine learning methods.

There are two main flavours of clustering methods: 
* hard clustering methods  
* soft clustering methods


In hard clustering each data point belongs to one and only one cluster, while in soft clustering a data point can belong to several different clusters with varying degrees of belonging.

We will apply one popular method for hard clustering (k-means) and one popular method for soft clustering (which is based on a probabilistic Gaussian mixture model) to a real-life application. In particular, consider you are running a Cafe in Helsinki and you want to segment costumers in order to design a new marketing strategy for the upcoming summer. Such a costumer segmentation can be done efficiently using clustering methods.

#### Code prerequisites

Before we start with the actual exercise let us first detail some helper functions for building the algorithms and understanding if they work right.

The following function will be used for reading the data from a csv file. 

In [ ]:
import csv
import numpy as np

def read_data(name):
    #Input: the name (address) of the file containing the data
    #Output: the data as a numpy array
    if 'csv' in name:
        data = []
        with open(name) as file:
            reader = csv.reader(file)
            for row in reader:
                data.append([float(x) for x in row]) 
        return np.asarray(data) 

The following function will be used for visualizing data points and clusters in ${\rm I\!R^{2}}$.  

In [ ]:
import matplotlib.pyplot as plt

def plotting(data, centroids=None, clusters=None):
    #Input: the data as an array, cluster means (centroids), cluster assignemnts in {0,1,...,k-1}   
    #Output: an scatter plot of the data in the clusters with cluster means
    plt.style.use('ggplot')
    plt.title("Data")
    plt.xlabel("feature x_1: customers' age")
    plt.ylabel("feature x_2: money spent during visit")
    
    if centroids is None and clusters is None:
        plt.scatter(data[:,0], data[:,1])
    if centroids is not None and clusters is None:
        plt.scatter(data[:,0], data[:,1])
        plt.scatter(centroids[:,0], centroids[:,1], marker="x", color="blue")
    if centroids is not None and clusters is not None:
        plt.scatter(data[:,0], data[:,1], c=clusters)
        plt.scatter(centroids[:,0], centroids[:,1], marker="x", color="blue")
    if centroids is None and clusters is not None:
        plt.scatter(data[:,0], data[:,1], c=clusters)
    
    plt.show()

## 2. Hard clustering with k-means
<a id="kmeans"></a>

A popular method for hard clustering is the k-means algorithm. K-means takes as input a list of data points $x^{(1)},...,x^{(N)}$ with $x^{(i)} \in {\rm I\!R^{d}}$ and groups it into $k$ non-overlapping clusters with cluster means $m_1,...,m_k$. As a hard-clustering method, k-means assigns each data point $x^{(i)}$ to exactly one cluster $y^{(i)}\in \{1,...,k\}$. 


K-means can be summarized as follows:

* Choose initial cluster means $ m_1,...,m_k \in {\rm I\!R^{d}}$

* Repeat until the clusters stop changing (or until another stopping condition):  

    * Assign each datapoint to a cluster whose mean is nearest. For all $i=1,...,N$, do  $$ y^{(i)} = \underset{c'}{\operatorname{argmin}} \|x^{(i)} - m_{c'}\|^2 $$
    * Check if clusters are active (have at least one member). For all $c=1,...,k$, set $$ b^{(c)} = \begin{cases} 1 & \mbox{ if } {\mid\{i: y^{(i)}= c\}\mid} > 0 \\ 0 & \mbox{ else.} \end{cases} $$
    * Set new cluster  means of active clusters by calculating the average of the points in an active cluster. For all $c=1,...,k$ with $b^{(c)}=1$, set $$ m_c = \frac{1}{\mid\{i: y^{(i)}= c\}\mid}{\sum_{i: y^{(i)}= c}x^{(i)}}$$ 


In summary, k-means consists of 4 steps:

* Step 1 - Initialize cluster means.
* Step 2 - Assign datapoints to the nearest cluster means.
* Step 3 - Set new cluster means.
* Step 4 - Put steps 2 and 3 in a loop.

### 2.1. Implementing k-means steps
<a id="steps_k-means"></a>

The Cafe owner wants to know if the customers form a uniform group or if there are several groups. In the latter case it might be useful to employ different marketing strategies targeting each group individually. So far the only information the owner collected was the age of the customers (there was a nasty incident in the past and now everyone is asked to show the ID when ordering alcoholic beverages) and the amount of money they spent (which can be recorded conveniently for card payments). The recordings are stored in the file "data.csv".

In the file "data.csv" you will find 51 rows, each of which contains one data point. First column contains the age of the customers and second column contains the amount of money they spent at the bar. Let's read it into a 51x2 numpy array and then visualize it using a scatter plot.

In [ ]:
data = read_data('data.csv')
plotting(data)

How many clusters do you see?

Now we can proceed to the main event.

#### STEP 1:   Choose centroids

Your task is to build a function that takes a dataset of $N$ data points ${x}^{(i)} \in \mathbb{R}^{d}$ and a number $k$ of clusters you want to partition your data into as inputs, and then randomly selects cluster centroids (means).

Tips:

- Even though right now we are looking at a 2-d data, in the end, your algorithm should work for any-dimensional data.
- You can use numpy.random.uniform for selecting random means from some interval of values
- It probably makes sense to specify the interval of means' coordinates close to the range of your data

In [ ]:
### STUDENT TASK ###
import numpy as np

def select_centroids(data, k):
    #INPUT: N x d data array, k number of clusters. 
    #OUTPUT: k x d array of k randomly assigned mean vectors with d dimensions.
    
    centroids = #...
    #...
    
    return centroids

Let's see how we're doing so far. Select 2 as the number of clusters and look at what/where they are.

Run the cell below for that.

In [ ]:
centroids = select_centroids(data, 2)
print(centroids)
plotting(data, centroids)

Makes sense? Let's go to Step 2 then.

#### STEP 2: Update cluster assignment 

In this step your task is to assign a data point $x^{(i)}$ to cluster $c$ with the nearest cluster mean $m_{c}$, measured by Euclidean distance $\| x^{(i)} - m_{c} \|$. The following function should return a vector of length $N$ whose entries are the cluster assignments $y^{(i)}$ for each point.

Tips:

- You can use _numpy.linalg.norm_ to measure the distance
- You can use _numpy.argmin_ to get the indices of the minimum values along an axis in an array

In [ ]:
### STUDENT TASK ###
import numpy as np

def assign_points(data, centroids): 
    #INPUT: N x d data array, k x d centroids array.
    #OUTPUT: N x 1 array of cluster assignments in {0,...,k-1}.
    
    clusters = #...
    #...
    
    return clusters

Let's look at what clusters we have so far.

Run the cell below for that.

In [ ]:
clusters = assign_points(data, centroids)
plotting(data, centroids, clusters)

Does it make sense? Yes? Awesome! Let's proceed further.

#### STEP 3: Update cluster centroids
Your task is to update the cluster means $m_{c}$ to be "in the middle" of the data points that were assigned to cluster c during the previous step. This "middle" is defined as the average (mean) of the data points belonging to the cluster. 

Tips:
- You can use numpy mean function to compute the mean
- You need to keep in mind that there might be a situation when none of the points were assigned to a cluster. You can either follow the algorithm in Section 2 or randomly reassign the cluster means in this case

In [ ]:
### STUDENT TASK ###
import numpy as np

def move_centroids(old_centroids, data, clusters):
    #INPUT:  N x d data array, k x d centroids array, N x 1 array of cluster assignments
    #OUTPUT: k x d array of relocated centroids
    
    new_centroids = #...
    #...
    
    return new_centroids

Let's see if the cluster means moved. Run the cell below for that.

In [ ]:
new_centroids = move_centroids(centroids, data, clusters)
plotting(data, new_centroids, clusters)

Are the cluster means in the middle of the clusters now? Yes? Nice! We're ready to assemble the whole algorithm now!

#### STEP 4: Putting together the pieces

Now it is time to combine the functions you have created so far into the final algorithm. The new part here is the loop where you repeat steps 2 and 3 until stopping criterion is fulfilled. Your task is to complete the following algorithm. Please print the final values of the centroids at the end of the algorithm. Furthermore, produce plots for the first three iterations of k-means.

Tip: 
- As stopping criterion we could use the requirement of the cluster assignments stop changing. Another option is to use a fixed number of iterations (here please use 10 iterations).

In [ ]:
### STUDENT TASK ###

def k_means(data, k):
    #INPUT: N x d data array, k number of clusters.
    #OUTPUT: N x 1 array of cluster assignments.
    
    #step 1
    centroids = select_centroids(data, k)
    
    #loop for steps 2 and 3
    #remember to plot the clustering results in the first 3 iterations
    #...
            
    return clusters

Let's see if our algorithm works. By now you know the drill: run the cell below.

In [ ]:
clusters = k_means(data, 2)
plotting(data, clusters=clusters)

Do you feel like the clusters are colored the way they should? How would you define the customer groups?

### 2.2. Handling local minima in k-means
<a id="local"></a>

As introduced in the course book, the k-means algorithm aims at minimizing the __empirical risk__: 

$$\varepsilon  ( \{m_{c}\}_{c=1}^{k},\{y^{(i)}\}_{i=1}^{N} \mid \{x^{(i)}\}_{i=1}^{N} )
=(1/N) \sum_{i=1}^{N} {\left\|x^{(i)}-m_{y^{(i)}}\right\|^2}
$$

Since the empirical risk is a highly non-convex function of the cluster means and assignments, the k-means method will sometimes get trapped in a local minimum. 

It is therefore useful to run k-means several times with different initializations for the cluster means and choose the cluster assignment that yields the smallest empirical risk. 

Your task is to implement a function that calculates the empirical risk $\varepsilon$, given the data $\{x^{(i)}\}_{i=1}^{N}$, cluster means (called centroids in the code bellow) $\{m_{c}\}_{c=1}^{k}$, and cluster assignments $\{y^{(i)}\}_{i=1}^{N}$.

Tip:
- You can use _numpy.linalg.norm_ to measure the Euclidean norm.

In [ ]:
### STUDENT TASK ###
import numpy as np

def empirical_risk(data, centroids, clusters):    
    #INPUT: N x d data array, k x d array of k mean vectors (centroids), 
    #       N x 1 array of cluster assignments.
    #OUTPUT: value of empirical risk
    
    risk = #...
    #...
    
    return risk

Your task is to improve the previous "k_means" algorithm by adding a loop and running k_means 50 times and collecting the cluster assignments it gives for each run and the corresponding empirical risk value. The output cluster should be the one with the lowest empirical risk. Print the lowest emprical risk. This time we will run the algorithm to find 3 clusters ($k=3$). 

In [ ]:
### STUDENT TASK ###
import numpy as np

def new_k_means(data, k):
    #INPUT: N x d data array, k number of clusters
    #OUTPUT: N x 1 array of the cluster assignment with the lowest empirical risk
    
    # initializing the array where we collect all cluster assignments  
    cluster_collection = np.zeros((50, data.shape[0]))
    # initializing the array where we collect all risk values 
    risk_collection = np.zeros(50)
    
    
    for i in range(50):
        #...
        
    #find the best cluster assignment and print the lowest found empirical risk
    #...
    
    return best_cluster

Let's see what we have now!

In [ ]:
best_cluster = new_k_means(data,3)
plotting(data, clusters=best_cluster)

## 3. Soft clustering with Gaussian Mixture Models (GMM) 
<a id="GMM"></a>

We are now interested in a more fine-grained analysis of the costumers. In particular, we would like to have some measure for the extend by which a costumer belongs to various groups. This is a soft-clustering problem where we associate each data point $x^{(i)}$(which represents a particular costumer) with a vector of membership $y^{(i)}= (y^{(i)}_1,...,y^{(i)}_k) \in [0,1]^k$, with $y^{(i)}_c$ representing the amount or confidence by which we assign $x^{(i)}$ to cluster c. 

Tip: 
- A refresher on basic concepts of probability theory (GMM, mean, covariance, probability density function) can be found at [this link](https://brilliant.org/wiki/gaussian-mixture-model/).

A principled approach to obtaining a soft-clustering method is based on interpreting the data points as realizations of a random variable. This random variable is distributed according to a mixture of Gaussian random vectors. Each component of this mixture represents a particular cluster c and is characterized by a mean vector $m_{c}$ and covariance matrix $C_{c}$. 

Suppose we have a dataset $x^{(1)},...,x^{(N)}$ with $x^{(i)} \in {\rm I\!R^{d}}$ and we want to cluster it into $k$ (overlapping) clusters. Thus, every data point $x^{(i)}$ will belong to each cluster $c$ with certain degrees. The degree $y^{(i)}_c$, of data point $x^{(i)}$ belonging to cluster $c$, is chosen as the probability that $x^{(i)}$ is generated by the $c$-th component $\mathcal{N}(m_{c},C_{c})$ of the GMM, i.e., 

$$y^{(i)}_c = \frac{\mathcal{N}(x^{(i)} ; m_c, C_c)}{\sum_{j=1}^k \mathcal{N}(x^{(i)} ; m_j, C_j)} $$


Here, $m$ is the estimate for the cluster mean and $C$ is the estimate for the covariance matrix. Given the degree of belongings $y^{(i)}_c$ we can then update the estimates for the cluster means and covariance matrix using the following equations: 

$$m_c = \frac{1}{N_c} \sum_{i=1}^N y^{(i)}_c x^{(i)} $$
$$C_c = \frac{1}{N_c} \sum_{i=1}^N y^{(i)}_c(x^{(i)} - m_c)(x^{(i)} - m_c)^T $$

, where $N_c = \sum_{i=1}^N y^{(i)}_c$.

The details of the algorithm can be found in the course book (see Algorithm 7 "A Soft-Clustering Algorithm"), For convenience, we provide here a general sketch of the algorithm:

1. Use an initial guess for GMM parameters (means and covariances)
2. Repeat for all the data, until convergence:
    1. Update degrees of belonging (all $y^{(i)}_c$ (see above))   
    2. Update GMM parameters (mean and covariances (see above))

So, let's compose it!

#### Understand the dataset

As in Section 2.1, we consider the dataset in "data.csv" file, which contains data points representing customers information. Let's plot the data again.

In [ ]:
data = read_data('data.csv')
plotting(data)

### 3.1. Implementing GMM clustering steps
<a id="steps_GMM"></a>

Let's implement each step of the soft-clustering algorithm as Python functions. Here, that the algorithm should output two clusters described by its parameters $m_1, m_2, C_1, C_2$

#### STEP 1: Initial guess for GMM parameters

Your task is to write a function for initializing the GMM parameters randomly from the dataset by setting the two mean vectors to two random data points and the two covariance matrices to the identity matrix. 

In [ ]:
### STUDENT TASK ###
import numpy as np

def initialize_parameters(data):
    #INPUT: N x d data array 
    #OUTPUT: random mean vectors of two clusters (m_1, m_2) 
    #        and unit covariance matrices (C_1, C_2)
    
    # get random feature vector from the dataset
    m_1 = #...
    m_2 = #...
    # unit covariance
    C_1 = #... 
    C_2 = #...
    return m_1, m_2, C_1, C_2

#### STEP 2: Update degrees of belonging $y^{(i)}$


Your task is to complete the function *update_degrees_of_belonging(data, m_1, m_2, C_1, C_2)* that reads the GMM parameters and returns two vectors $y_1$ and $y_2$, where each one describes the belonging degree of all datapoints to each cluster. For example, for cluster 1, $y_1$ is a N by 1 vector where the $i^{th}$ element is:

$$y^{(i)}_1 = \frac{\mathcal{N}(x^{(i)} ; m_1, C_1)}{\mathcal{N}(x^{(i)} ; m_1, C_1) + \mathcal{N}(x^{(i)} ; m_2, C_2)} $$

Tip:
- You can use multivariate_normal.pdf() function in scipy.stats library to compute the density.

In [ ]:
### STUDENT TASK ###
import numpy as np

def update_degrees_of_belonging(data, m_1, m_2, C_1, C_2): 
    #INPUT: data, m_1, m_2, C_1, C_2  (dataset and GMM parameters)
    #OUTPUT: y_1, y_2 as vector with the belonging degree of every data to cluster 1 and 2
    
    #...
    
    return(y_1,y_2)

#### STEP 3: Update GMM parameters of one cluster


Your task is to complete the function *update_GMM_pars(data, y)* that receives the data and degrees of belonging to one cluster  and returns the updated GMM parameters for that cluster according to 

$$m = \frac{1}{M} \sum_{i=1}^N y^{(i)}x^{(i)} $$
$$C = \frac{1}{M} \sum_{i=1}^N y^{(i)}(x^{(i)} - m)(x^{(i)} - m)^T $$
, where $M = \sum_{i=1}^N y^{(i)}$ of every $y$ vector



In [ ]:
### STUDENT TASK ###
import numpy as np

def update_GMM_pars(data, y): 
    #INPUT: data, y (dataset and the belonging degree)
    #OUTPUT: m_new, C_new
    
    #...
    return(m_new,C_new)

#### STEP 4: Putting together the pieces

Now, let's put the pieces together and plot the data points and clusters. The following code uses the previous functions to go through the steps of the clustering algorithm. Execute the following code to plot in the same graph the dataset, to which cluster they belong and the clusters normal distribution. The code should produce the plot if the previous functions are implemented correctly. You do not need to implement anything new at this stage.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

num_iterations = 50
data = read_data('data.csv')

# Step 1: 
m_1, m_2, C_1, C_2 = initialize_parameters(data)

for it in range(num_iterations):
    # Step 2:
    y_1, y_2 = update_degrees_of_belonging(data, m_1, m_2, C_1, C_2)
    # Step 3:
    m_1, C_1 = update_GMM_pars(data, y_1)
    m_2, C_2 = update_GMM_pars(data, y_2)

print("Mean 1: " + str(m_1) + ", Mean 2: " + str(m_2))
print("Cov 1: \n" + str(C_1))
print("Cov 2: \n" + str(C_2))

clusters = np.argmax(np.vstack((y_1,y_2)),axis=0)
plt.scatter(data[:,0], data[:,1], c=clusters, s=13)

#Visualization of results
x_plot = np.linspace(-9,3, 100)
y_plot = np.linspace(-3,7, 100)

x_1_mesh, y_1_mesh = np.meshgrid(x_plot, y_plot)
z_1 = plt.mlab.bivariate_normal(x_1_mesh, y_1_mesh, np.sqrt(C_1[0, 0]), \
                                np.sqrt(C_1[1, 1]), m_1[0], m_1[1])
plt.contour(x_1_mesh , y_1_mesh , z_1,4,colors='red')
x_2_mesh, y_2_mesh = np.meshgrid(x_plot, y_plot)
z_2 = plt.mlab.bivariate_normal(x_2_mesh, y_2_mesh, np.sqrt(C_2[0, 0]), \
                                np.sqrt(C_2[1, 1]), m_2[0], m_2[1])
plt.contour(x_2_mesh , y_2_mesh , z_2,4,colors='red')

plt.scatter( [m_1[0]], [m_1[1]], marker='x',c='red')
plt.scatter( [m_2[0]], [m_2[1]], marker='x',c='red')
plt.title("Soft clustering with GMM")
plt.xlabel("feature x_1: customers' age")
plt.ylabel("feature x_2: money spent during visit")
plt.show()